# Rotation Data Analysis Using OLS - Slicing method
OLS-Slicing method first slice the raw data into several pieces to reduce memory requirement. Then perform OLS on each piece.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib.ticker import ScalarFormatter

# Reading Raw Data
Reading data method is the same as DLS

In [2]:
def read_data(location_file, local_time, rotation_frequency):
    """
    This function reads the raw data file, whose pathname are stored in the location_file.
    Return: DataFrame obejct that contains all the raw data.
    local_time (s) should be added to the time tag so that it starts from the local sidereal frame. 
    """
    # read locations of data file
    with open(location_file) as f:
        files = f.readlines()
    files = [x.strip() for x in files]  # remove the '\n' in each line.
    
    # read and concatenate each data file
    data_df = pd.DataFrame()  # for fast reading and storing data
    for location in files:
        data = pd.read_csv(location, sep='\t', header=None, index_col=False)
        data_df = pd.concat([data_df, data])  # concatenate each data
    
    # change time to local sidereal time
    data_df[1] = data_df[1] + local_time  
    
    # change voltage to fractional frequency [unitless]
    factor_v2f = 6.65726 * 1e-8  # calibration factor when rotation frequency is 360
    data_df[3] = data_df[3] * factor_v2f * rotation_frequency / 360
    
    return data_df

# OLS function
Ordinary Least Square(OLS) method uses the most simple regression method: Linear Least Square Regression. See DLS for the explanation of it.

In this script, we use OLS to fit part of the frequency components. They are shown below:

\begin{align*}
\frac{\delta f}{f} = A + &S_{2\omega_R}sin(2\omega_Rt) + C_{2\omega_R}cos(2\omega_Rt)\\
                       + &S_{2\omega_R + \omega_\oplus t}sin(2\omega_Rt + \omega_\oplus t) + C_{2\omega_R + \omega_\oplus t}cos(2\omega_Rt + \omega_\oplus t)\\
                       + &S_{2\omega_R - \omega_\oplus t}sin(2\omega_Rt - \omega_\oplus t) + C_{2\omega_R - \omega_\oplus t}cos(2\omega_Rt - \omega_\oplus t)]\\
                       + &S_{2\omega_R + 2\omega_\oplus t}sin(2\omega_Rt + 2\omega_\oplus t) + C_{2\omega_R + 2\omega_\oplus t}cos(2\omega_Rt + 2\omega_\oplus t)\\
                       + &S_{2\omega_R - 2\omega_\oplus t}sin(2\omega_Rt - 2\omega_\oplus t) + C_{2\omega_R - 2\omega_\oplus t}cos(2\omega_Rt - 2\omega_\oplus t)
\end{align*}

For simplicity, we express it as:

\begin{equation}
Y = \beta_0 + \beta_1X_1 + \beta_2X_2 + \beta_3X_3\ ...
\end{equation}


*Output / Input requires the same as demodulate function in script DLS*

In [4]:
def OLS(dataset, update=0):
    """
    Given a dataset of signal which should be a numpy array with the form [index, sidereal_time, angle, delta_f], fitting it by equation (stated in OLS section). 
    Return parameters include [theta, standard deviation, averaged time]
    """
    # initialization
    N = dataset[:,0]
    t = dataset[:,1]
    phi_R = dataset[:,2]
    delta_f = dataset[:,3]
    
    # pre-calculation
    w_s = 2 * np.pi / (23*3600 + 56*60 + 4) # sidereal frequency
    phi_S = w_s * t  # sidereal phase
    
    # creating matrix X
    X = np.zeros([dataset.shape[0],11])
    X[:,0] = 1
    X[:,1] = np.sin(2*phi_R)
    X[:,2] = np.cos(2*phi_R)
    
    X[:,3] = np.sin(2*phi_R + phi_S)
    X[:,4] = np.cos(2*phi_R + phi_S)
    
    X[:,5] = np.sin(2*phi_R - phi_S)
    X[:,6] = np.cos(2*phi_R - phi_S)
    
    X[:,7] = np.sin(2*phi_R + 2*phi_S)
    X[:,8] = np.cos(2*phi_R + 2*phi_S)
    
    X[:,9] = np.sin(2*phi_R - 2*phi_S)
    X[:,10] = np.cos(2*phi_R - 2*phi_S)
    
    
    # perform OLS
    varcov = np.linalg.inv(np.dot(X.T, X))  # inverted inner product of X
    beta = np.dot(varcov, np.dot(X.T,delta_f))  # fitting parameters
    
    res = np.inner(beta.T, X) - delta_f  # residual
    dres = np.std(res)  # standard deviation of the residual
    
    V_beta = dres * dres * varcov  # variance matrix of fitting parameters
    sigma_beta = np.sqrt(np.diag(V_beta))  # standard deviation for fitting parameters
    
    # calculate the mean time for all these parameters
    t_mean = np.mean(t)  # average time for the dataset
    
    # calculate the number of samples inside the dataset
    N_sample = np.size(N)
    
    # packing all parameters in one vector
    params = pd.DataFrame([[beta, sigma_beta, t_mean, N_sample]],columns=['beta','sigma','time','Nsamples'])
    
    return params
    

In [5]:
def select_param(params_df, param):
    """
    Given a params dataframe, select a certain param. Output the result as 2D narray
    """
    param_df = params_df[param]  # choose a parameter
    param_na = param_df.values  # change it to narray
    param_na = np.array(param_na.tolist())  # change it to 2D narray (tricky part)
    
    return param_na

# Function for Slicing dataset
Slice the file into pieces. Do OLS in each piece. Visualize them in the end.

In [22]:
def ols_slicing(data_df,N_slice):
    """
    input data_df contains the raw data.
    N_slice is the number of subdata you want to have.
    
    output params that contains parameters for each subdata.
    """
    
    # slice 
    N_data = data_df.shape[0]
    N_piece = int(N_data / N_slice)
    params = pd.DataFrame()  # for storing params in each sub data

    for i in range(N_slice):
        # slicing each sub_data
        start = i * N_piece
        stop = (i+1) * N_piece
        next_stop = (i+2) * N_piece
        if next_stop > N_data:
            sub_data_df = data_df[start:]
        else:
            sub_data_df = data_df[start:stop]

        # do OLS on each sub_data
        sub_data = sub_data_df.values  # change to numpy array
        sub_param = OLS(sub_data)
        params = pd.concat((params, sub_param))

    params.reset_index(drop='True')
    return params

CPU times: user 1min 26s, sys: 8.35 s, total: 1min 34s
Wall time: 1min 21s


## Parameter table
|Parameter|A|$S_{2\omega_R}$|$C_{2\omega_R}$|$S_{2\omega_R+\omega_\oplus}$|$C_{2\omega_R+\omega_\oplus}$|$S_{2\omega_R-\omega_\oplus}$|$C_{2\omega_R-\omega_\oplus}$|$S_{2\omega_R+2\omega_\oplus}$|$C_{2\omega_R+2\omega_\oplus}$|$S_{2\omega_R-2\omega_\oplus}$|$C_{2\omega_R-2\omega_\oplus}$|
|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|
|$\beta$|$\beta_0$|$\beta_1$|$\beta_2$|$\beta_3$|$\beta_4$|$\beta_5$|$\beta_6$|$\beta_7$|$\beta_8$|$\beta_9$|$\beta_{10}$|
|*index_vi*|0|1|2|3|4|5|6|7|8|9|10|
As defined above, $\beta$ is just another name for parameter $S_{...}$ or $C_{...}$. 

$\beta$ is used in fitting, $S_{...}$ and $C_{...}$ are used in physics related equation.

# Sum up Slices
we obtained value, $A_i$, and standard deviation, $\sigma_i$ for each parameter. $i \in$ [0, $N_S$) stands for different subsets. $N_S$ is the value of *Slices*. 

We can get a more accurate parameter {$A_{tot}$ and $\sigma_{tot}$} by adding results from all subsets up.
\begin{align*}
    \frac{1}{\sigma_{tot}^2} &= \sum_{i}^{N_S}\frac{1}{\sigma_i^2}  \\
    A_{tot} &= (\sum_{i}^{N_S}\frac{A_i}{\sigma_i^2})\sigma_{tot}^2
\end{align*}

This method assumes that different subsets don't correlate to each other (measurement result doesn't depend on each other).

## Class *var*
Because these parameters need to be treated differently than the normal scaler. I developed a *var* class for easy handling variables with value and standard deviation.

Objects belong to *var* class would have different rules of calculation. They are defined below:

For variable $X_1$ and $X_2$. Their value and standard deviation are $x_1$, $x_2$, $\sigma_1$, and $\sigma_2$. Their addition, $X = X_1 + X_2$, follows:
\begin{align*}
\frac{1}{\sigma^2} &= \frac{1}{\sigma^2_1} + \frac{1}{\sigma^2_2} \\
x &= (\frac{x_1}{\sigma_1^2} + \frac{x_2}{\sigma_2^2})\sigma^2
\end{align*}

For a variable $X$ times a constant $a$, resulting new variable $X'$:
\begin{align*}
x' &= ax\\
\sigma' &= |a|\sigma
\end{align*}

In [26]:
class var():
    """
    This class is for calculating the addition between variables with standard deviation, sigma.
    Bug: couldn't perform division. To divide a constant, please multiply its inverse. 
    Also the multiplication between variables is not supported. We don't need them.
    """
    def __init__(self, value, sigma):
        self.value = value
        self.sigma = sigma
        
    def __add__(self, other):
        if isinstance(other, var):
            sum_sigma = 1 / (np.sqrt(1 / self.sigma**2 + 1 / other.sigma**2))
            sum_value = (self.value / self.sigma**2 + other.value / other.sigma**2) * sum_sigma**2
            return var(sum_value, sum_sigma)
        else:
            print('Operation not supported!')
            return None
        
    def __sub__(self, other):
        sub_other = var(-other.value, other.sigma)
        
        return self.__add__(sub_other)
    def __mul__(self, other):
        if isinstance(other, (int, float)):  # if multiplied by a scalar
            return var(self.value * other, self.sigma * abs(other))
        else:
            print('Operation not supported!')
            return None
        
    def __turediv__(self, other):  # devision doesn't work
        return __mul__(self, 1 / other)
    
    __rmul__ = __mul__  # reverse multiply still holds
            

*param_sum* is used to stored the summed up result. It's a vector with length of the number of parameters. One position stores one parameter.


In [19]:
def add_params(params):
    """
    This function recieves the output parameter dataframe from function slice.
    It returns a var object that contains all summed up parameters.
    """
    # initilization
    beta = select_param(params, 'beta')
    sigma = select_param(params, 'sigma')
    
    # create var object
    N_params = 11
    param_sum = [None] * N_params
    for i in range(N_params):
        # for wach parameter
        beta_vi = beta[:, i]
        sigma_vi = sigma[:, i]
        # sum different pieces of one parameter up (with sigma)
        param_sum[i] = var(beta_vi[0], sigma_vi[0])
        for j in range(1, beta_vi.shape[0]):
            param_sum[i] = param_sum[i] + var(beta_vi[j], sigma_vi[j])
            
    return param_sum

# Calculate EMS coefficients
## parameter index table
|*param_sum*|A|$S_{2\omega_R}$|$C_{2\omega_R}$|$S_{2\omega_R+\omega_\oplus}$|$C_{2\omega_R+\omega_\oplus}$|$S_{2\omega_R-\omega_\oplus}$|$C_{2\omega_R-\omega_\oplus}$|$S_{2\omega_R+2\omega_\oplus}$|$C_{2\omega_R+2\omega_\oplus}$|$S_{2\omega_R-2\omega_\oplus}$|$C_{2\omega_R-2\omega_\oplus}$|
|:-:|
|*i*|0|1|2|3|4|5|6|7|8|9|10|

Please refer to Francis's notebook for the fomular to calculate EMS coefficients from these parameters.

In [21]:

def cal_co(param_sum):
    """
    This function calculates the EMS coefficient from the param_sum variable.
    """
    X = 58.014 * 2 * np.pi / 360  # colatitude of the laborotory (radius)
    sin_X = np.sin(X)
    cos_X = np.cos(X)

    # calculate the coefficients of EMS
    c_T_Q =  1 / (-4 * sin_X**2) * param_sum[2]
    c_T_X_1 = 1 / (-4 * (1 + cos_X) * sin_X) * param_sum[3]
    c_T_X_2 = 1 / (4 * (cos_X - 1) * sin_X) * param_sum[5]
    c_T_Y_1 = 1 / (-4 * (1 + cos_X) * sin_X) * param_sum[4]
    c_T_Y_2 = 1 / (4 * (cos_X - 1) * sin_X) * param_sum[6]
    c_T_Z = 1 / (2 * (1 + cos_X)**2) * param_sum[7]
    c_T_M = 1 / (2 * (cos_X - 1)**2) * param_sum[10]
    c_T_u = 1 / (2 * (1 + cos_X)**2) * param_sum[8]

    c_T_X = c_T_X_1 + c_T_X_2
    c_T_Y = c_T_Y_1 + c_T_Y_2
    
    return [c_T_Q, c_T_X, c_T_Y, c_T_Z, c_T_M, c_T_u]